In [1]:
%matplotlib inline
from pylab import *
import cv2
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
rcParams['figure.figsize'] = 10, 10

In [3]:
from dataset import load_image

In [4]:
import torch

In [5]:
from utils import cuda

In [6]:
from generate_masks import get_model

In [7]:
from albumentations import Compose, Normalize

In [8]:
from albumentations.pytorch.functional import img_to_tensor

In [9]:
def img_transform(p=1):
    return Compose([
        Normalize(p=1)
    ], p=p)


In [10]:
def mask_overlay(image, mask, color=(0, 255, 0)):
    """
    Helper function to visualize mask on the top of the car
    """
    mask = np.dstack((mask, mask, mask)) * np.array(color)
    mask = mask.astype(np.uint8)
    weighted_sum = cv2.addWeighted(mask, 0.5, image, 0.5, 0.)
    img = image.copy()
    ind = mask[:, :, 1] > 0    
    img[ind] = weighted_sum[ind]    
    return img

In [11]:
model_path = 'data/models/unet16_parts_20/model_0.pt'
model = get_model(model_path, model_type='UNet16', problem_type='parts')

RuntimeError: Error(s) in loading state_dict for UNet16:
	Missing key(s) in state_dict: "center.block.1.conv.weight", "center.block.1.norm_act.running_var", "center.block.1.norm_act.running_mean", "center.block.1.norm_act.weight", "center.block.1.norm_act.bias", "center.block.2.conv.weight", "center.block.2.norm_act.running_var", "center.block.2.norm_act.running_mean", "center.block.2.norm_act.weight", "center.block.2.norm_act.bias", "dec5.block.1.conv.weight", "dec5.block.1.norm_act.running_var", "dec5.block.1.norm_act.running_mean", "dec5.block.1.norm_act.weight", "dec5.block.1.norm_act.bias", "dec5.block.2.conv.weight", "dec5.block.2.norm_act.running_var", "dec5.block.2.norm_act.running_mean", "dec5.block.2.norm_act.weight", "dec5.block.2.norm_act.bias", "dec4.block.1.conv.weight", "dec4.block.1.norm_act.running_var", "dec4.block.1.norm_act.running_mean", "dec4.block.1.norm_act.weight", "dec4.block.1.norm_act.bias", "dec4.block.2.conv.weight", "dec4.block.2.norm_act.running_var", "d

In [ ]:
img_file_name = 'data/cropped_train/instrument_dataset_3/images/frame004.jpg'
gt_file_name = 'data/cropped_train/instrument_dataset_3/binary_masks/frame004.png'

In [ ]:
image = load_image(img_file_name)
gt = cv2.imread(gt_file_name, 0) > 0

In [ ]:
imshow(image)

In [ ]:
with torch.no_grad():
    input_image = torch.unsqueeze(img_to_tensor(img_transform(p=1)(image=image)['image']).cuda(), dim=0)

In [ ]:
mask = model(input_image)

In [ ]:
mask_array = mask.data[0].cpu().numpy()[0]

In [ ]:
imshow(mask_array > 0)

In [ ]:
imshow(mask_overlay(image, (mask_array > 0).astype(np.uint8)))